### Задание
Использовать полученные знания в части обучения моделей для обучения и подбора параметров в задачах из предыдущих ДЗ на выбор (мфо, задача с тем возьмёт ли человек кредит или нет)  
В частности использовать:  


- GaussianNB
- LogisticRegression 
- Linear + PolynomialFeatures.
- RandomForestClassifier 
- При обучении использовать make_pipeline
- GridSearchCV для поиска параметров 
- Посчитать матрики (f1 + точность + AUC). 

#### Анализ задания

- Подготовим максимально качественные данные для наших моделей. Используем отфильтрованные данные из 5 задания и, если потребуется, дополнительно обработаем их.
- Используем 4 модели машинного обучения из библиотеки sklearn: Gaussian Naive Bayes, Logistic Regression, Linear + Polynomial Features, Random Forest Classifier
- Для каждой модели проведем поиск по сетке параметров (Grid Search) для нахождения оптимальных гиперпараметров.
- Оценим качество моделей с помощью метрик F1 Score, Accuracy и AUC.
- Сравним результаты всех моделей и выберем наилучшую для дальнейшего использования.


### Загрузка и предобработка данных

Загрузим предобработанные в 5 уроке данные

In [1]:
import pandas as pd
credit_train_df = pd.read_csv('credit_train_cleared.csv')
print(credit_train_df.head())
print("\nКоличество строк и столбцов:")
print(credit_train_df.shape)

   gender       age  marital_status  job_position  credit_sum  credit_month  \
0     0.0  0.150943            0.75          0.75    0.227807      0.212121   
1     1.0  0.150943            0.50          0.75    0.070015      0.333333   
2     1.0  0.339623            0.25          0.75    0.099553      0.212121   
3     1.0  0.245283            0.50          0.75    0.247503      0.212121   
4     0.0  0.150943            0.75          0.75    0.121157      0.212121   

   tariff_id  score_shk  education  monthly_income  credit_count  \
0        1.1   0.532554       0.25        0.058201      0.157895   
1        1.0   0.462837       0.75        0.032804      0.105263   
2        1.4   0.501961       0.75        0.047619      0.210526   
3        1.1   0.307946       0.25        0.052910      0.052632   
4        1.1   0.335454       0.25        0.021164      0.052632   

   overdue_credit_count  open_account_flg  region_КРАСНОДАРСКИЙ КРАЙ  \
0                   0.0                 0   

Данные в столбце 'open_account_flg' не симметричны. Удалим лишние данные, чтобы количество выданных и не выданных кредитов было равным.

In [2]:
from sklearn.model_selection import train_test_split

value_counts = credit_train_df['open_account_flg'].value_counts()
count_0 = value_counts.get(0, 0)
count_1 = value_counts.get(1, 0)
num_to_remove = count_0 - count_1

credit_train_df_balanced = pd.concat([
    credit_train_df[credit_train_df['open_account_flg'] == 1],
    credit_train_df[credit_train_df['open_account_flg'] == 0].sample(n=count_1, random_state=0)
])

X = credit_train_df_balanced.drop(columns=['open_account_flg'])
y = credit_train_df_balanced['open_account_flg']

Разобьем наши данные на тестовую и валидационную выборку

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=0)

### Использование базовых моделей

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

def train_and_evaluate(model, param_grid, X_train, y_train, X_val, y_val, scoring='f1', threshold=None):
    grid_search = GridSearchCV(model, param_grid, cv=10, scoring=scoring, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    print("Лучшие параметры:", best_params)

    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)

    y_pred = best_model.predict(X_val)
    
    if threshold is not None:
        y_pred_proba = y_pred
        y_pred = (y_pred > threshold).astype(int)
    else:
        y_pred_proba = best_model.predict_proba(X_val)[:, 1]

    f1 = f1_score(y_val, y_pred)
    accuracy = accuracy_score(y_val, y_pred)
    auc = roc_auc_score(y_val, y_pred_proba)

    print(f"F1 Score: {f1}")
    print(f"Accuracy: {accuracy}")
    print(f"AUC: {auc}")

#### GaussianNB

In [5]:
gnb_model = GaussianNB()

param_grid_gnb_model = {
    'var_smoothing': [1e-11, 1e-10, 1e-9, 1e-8, 1e-7]
}

train_and_evaluate(gnb_model, param_grid_gnb_model, X_train, y_train, X_val, y_val)

Лучшие параметры: {'var_smoothing': 1e-11}
F1 Score: 0.5955882352941176
Accuracy: 0.5849056603773585
AUC: 0.6213308047493404


#### LogisticRegression

In [6]:
lr_model = LogisticRegression(max_iter=10000)

param_grid_lr_model = [
    {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1'],
        'solver': ['liblinear', 'saga']
    },
    {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l2'],
        'solver': ['liblinear', 'lbfgs', 'sag', 'saga', 'newton-cg', 'newton-cholesky']
    },
    {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['elasticnet'],
        'solver': ['saga'],
        'l1_ratio': [0.1, 0.5, 0.9, 0.99]
    }
]

train_and_evaluate(lr_model, param_grid_lr_model, X_train, y_train, X_val, y_val)

Лучшие параметры: {'C': 1, 'l1_ratio': 0.9, 'penalty': 'elasticnet', 'solver': 'saga'}
F1 Score: 0.6132665832290363
Accuracy: 0.6113207547169811
AUC: 0.6492192256951491


#### PolynomialFeatures + LinearRegression

In [7]:
poly_lin_model = Pipeline([
    ('poly', PolynomialFeatures()),
    ('linear', LinearRegression())
])

param_grid_poly_lin_model = {
    'poly__degree': [1, 2, 3, 4]
}

train_and_evaluate(poly_lin_model, param_grid_poly_lin_model, X_train, y_train, X_val, y_val, scoring='neg_mean_squared_error', threshold=0.5)

Лучшие параметры: {'poly__degree': 2}
F1 Score: 0.630272952853598
Accuracy: 0.6251572327044025
AUC: 0.6738982900345037


#### RandomForest

In [8]:
rf_model = RandomForestClassifier(random_state=0)

param_grid_rf_model = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30, 50, 100],
    'min_samples_split': [2, 5, 10, 20, 50],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

train_and_evaluate(rf_model, param_grid_rf_model, X_train, y_train, X_val, y_val)

Лучшие параметры: {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 20, 'n_estimators': 100}
F1 Score: 0.6782608695652174
Accuracy: 0.6742138364779874
AUC: 0.7312290694134362


### Выводы
- Были построены и оценены четыре модели машинного обучения: Gaussian Naive Bayes, Logistic Regression, PolynomialFeatures + LinearRegression и Random Forest Classifier. 
- Для каждой модели был проведен поиск по сетке для нахождения оптимальных гиперпараметров и оценены метрики F1 Score, Accuracy и AUC.
- Наилучшей моделью для данной задачи оказался Random Forest Classifier, который показал лучшие значения всех метрик.
